In [16]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords, words
from wordcloud import WordCloud

# Download necessary NLTK packages
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('words')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\andre\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\andre\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\andre\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\andre\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\andre\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [17]:
df2 = pd.read_csv('../dataset/DataSetGitHub/train.csv')
df2.head()

,Category,Text
0,Accountant,education omba executive leadership university...
1,Accountant,howard gerrard accountant deyjobcom birmingham...
2,Accountant,kevin frank senior accountant inforesumekraftc...
3,Accountant,place birth nationality olivia ogilvy accounta...
4,Accountant,stephen greet cpa senior accountant 9 year exp...


In [18]:
df1 = pd.read_csv('../dataset/DataSetKaggle/Resume/Resume.csv')
df1.head()

,ID,Resume_str,Resume_html,Category
0,16852973,HR ADMINISTRATOR/MARKETING ASSOCIATE\...,"<div class=""fontsize fontface vmargins hmargin...",HR
1,22323967,"HR SPECIALIST, US HR OPERATIONS ...","<div class=""fontsize fontface vmargins hmargin...",HR
2,33176873,HR DIRECTOR Summary Over 2...,"<div class=""fontsize fontface vmargins hmargin...",HR
3,27018550,HR SPECIALIST Summary Dedica...,"<div class=""fontsize fontface vmargins hmargin...",HR
4,17812897,HR MANAGER Skill Highlights ...,"<div class=""fontsize fontface vmargins hmargin...",HR


In [19]:
df2.info()
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13389 entries, 0 to 13388
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Category  13389 non-null  object
 1   Text      13389 non-null  object
dtypes: object(2)
memory usage: 209.3+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2484 entries, 0 to 2483
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ID           2484 non-null   int64 
 1   Resume_str   2484 non-null   object
 2   Resume_html  2484 non-null   object
 3   Category     2484 non-null   object
dtypes: int64(1), object(3)
memory usage: 77.8+ KB


In [20]:
df1.drop(columns = ['Resume_html', 'ID'], inplace = True)
df1.rename(columns={'Resume_str': 'Text'}, inplace=True)
df1.head()

,Text,Category
0,HR ADMINISTRATOR/MARKETING ASSOCIATE\...,HR
1,"HR SPECIALIST, US HR OPERATIONS ...",HR
2,HR DIRECTOR Summary Over 2...,HR
3,HR SPECIALIST Summary Dedica...,HR
4,HR MANAGER Skill Highlights ...,HR


In [21]:
print(df2['Category'].unique())

['Accountant' 'Advocate' 'Agriculture' 'Apparel' 'Architecture' 'Arts'
 'Automobile' 'Aviation' 'Banking' 'Blockchain' 'BPO'
 'Building and Construction' 'Business Analyst' 'Civil Engineer'
 'Consultant' 'Data Science' 'Database' 'Designing' 'DevOps'
 'Digital Media' 'DotNet Developer' 'Education' 'Electrical Engineering'
 'ETL Developer' 'Finance' 'Food and Beverages' 'Health and Fitness'
 'Human Resources' 'Information Technology' 'Java Developer' 'Management'
 'Mechanical Engineer' 'Network Security Engineer' 'Operations Manager'
 'PMO' 'Public Relations' 'Python Developer' 'React Developer' 'Sales'
 'SAP Developer' 'SQL Developer' 'Testing' 'Web Designing']


In [22]:
print(df1['Category'].unique())

['HR' 'DESIGNER' 'INFORMATION-TECHNOLOGY' 'TEACHER' 'ADVOCATE'
 'BUSINESS-DEVELOPMENT' 'HEALTHCARE' 'FITNESS' 'AGRICULTURE' 'BPO' 'SALES'
 'CONSULTANT' 'DIGITAL-MEDIA' 'AUTOMOBILE' 'CHEF' 'FINANCE' 'APPAREL'
 'ENGINEERING' 'ACCOUNTANT' 'CONSTRUCTION' 'PUBLIC-RELATIONS' 'BANKING'
 'ARTS' 'AVIATION']


### Merge Dataset

In [23]:
map_category = {
    'ACCOUNTANT': 'Accountant',
    'ADVOCATE': 'Advocate',
    'AGRICULTURE': 'Agriculture',
    'APPAREL': 'Apparel',
    'ARTS': 'Arts',
    'AUTOMOBILE': 'Automobile',
    'AVIATION': 'Aviation',
    'BANKING': 'Banking',
    'BPO': 'BPO',
    'CONSTRUCTION': 'Building and Construction',
    'CONSULTANT': 'Consultant',
    'DIGITAL-MEDIA': 'Digital Media',
    'SALES': 'Sales',
    'FINANCE': 'Finance',
    'PUBLIC-RELATIONS': 'Public Relations',
    'INFORMATION-TECHNOLOGY': 'Information Technology',
    'TEACHER': 'Education',
    'HR': 'Human Resources',
    'DESIGNER': 'Designing',
    'BUSINESS-DEVELOPMENT': 'Business Analyst',
    'HEALTHCARE': 'Health and Fitness',
    'FITNESS': 'Health and Fitness',
    'ACCOUNTANT': 'Accountant',
    'ARTS': 'Arts',

    'ENGINEERING': 'Engineer',  #new class
    'CHEF': 'Chef'              #new class
}


In [24]:
df1['Category'] = df1['Category'].map(map_category)
df = pd.concat([df2, df1], ignore_index=True)

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15873 entries, 0 to 15872
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Category  15873 non-null  object
 1   Text      15873 non-null  object
dtypes: object(2)
memory usage: 248.1+ KB


## Preprocessing

In [26]:
# Get the list of valid English words
english_words = set(words.words())

# Get a list of stop word
stop_words = set(nltk.corpus.stopwords.words('english'))

# Initialize the stemmer
stemmer = PorterStemmer()

# Initialize the lemmatizer
lemmatizer = WordNetLemmatizer()

In [27]:
def preprocess_text(text, remove_numbers=True, use_dictionary=True, use_lemmatization=False, use_stemming=True):
    """
    Preprocesses the text: 
    - Converts to lowercase
    - Removes punctuation
    - Tokenizes
    - Removes stopwords
    - Removes words with numbers (if enabled)
    - Uses dictionary filtering (if enabled)
    - Applies lemmatization or stemming based on parameters
    """
    
    text = text.lower()                         # Convert to lowercase
    text = re.sub(r'[^\w\s]', ' ', text)         # TODO non ho capito perchè commantando diminuiscono i token     # Remove all punctuation  
    text = re.sub(r'\s+', ' ', text).strip()    # Remove multiple spaces
    
    tokens = word_tokenize(text) # Tokenize the text

    processed_tokens = []
    
    for token in tokens:
        # Remove words containing numbers (if enabled)
        if remove_numbers and any(char.isdigit() for char in token):
            continue

        # Dictionary filtering (if enabled)
        if use_dictionary and token not in english_words:
            continue

        # Remove stopwords
        if token in stop_words:
            continue

        if use_lemmatization:
            token = lemmatizer.lemmatize(token)
        elif use_stemming:
            token = stemmer.stem(token)

        processed_tokens.append(token)
        
        # Check if the token contains only letters/numbers
        # if re.fullmatch(r'[a-zA-Z0-9]+', token):              # not usefull if use_dictionary = True

        # Exclude stopwords
        # if token not in stop_words and not token.isdigit():
        #     # If requested, check if it is in the English dictionary or is a number
        #     if use_dictionary:
        #         if token in english_words:   # TODO probably, in classification, it's not necessary to have the digits
        #             # stemmed = stemmer.stem(token)
        #             lemmed = lemmatizer.lemmatize(token)      # TODO try lemmatizer instead stemmer, it could improve performance
        #             processed_tokens.append(lemmed)
        #     else:
        #         # stemmed = stemmer.stem(token)
        #         lemmed = lemmatizer.lemmatize(token)          # TODO try lemmatizer instead stemmer, it could improve performance
        #         processed_tokens.append(lemmed)
        # # If the token is not valid, discard it

    
    #tokens = remove_non_english(tokens)    # Remove non-English words
    
    return ' '.join(processed_tokens)


In [28]:
from tqdm import tqdm  # Library for the progress bar

def preprocess_text_with_progress(df, column_name, **kwargs):
    """
    Applies preprocess_text to a DataFrame column with a progress bar.
    
    Args:
    - df (pd.DataFrame): DataFrame containing the data
    - column_name (str): Name of the column to preprocess
    - kwargs: Parameters to pass to the preprocess_text function
    
    Returns:
    - pd.Series: Series containing the preprocessed texts
    """
    tqdm.pandas(desc="Processing Resumes")  # Enable tqdm for Pandas
    return df[column_name].progress_apply(lambda x: preprocess_text(x, **kwargs))

In [29]:
import itertools
import os
parameter_sets = [
    #{"remove_numbers": True, "use_dictionary": False, "use_lemmatization": True, "use_stemming": False},
    #{"remove_numbers": True, "use_dictionary": False, "use_lemmatization": False, "use_stemming": True},
    #{"remove_numbers": False, "use_dictionary": True, "use_lemmatization": True, "use_stemming": False},
    #{"remove_numbers": False, "use_dictionary": True, "use_lemmatization": False, "use_stemming": True},
    {"remove_numbers": True, "use_dictionary": True, "use_lemmatization": True, "use_stemming": False},
    #{"remove_numbers": True, "use_dictionary": True, "use_lemmatization": False, "use_stemming": True},
    #{"remove_numbers": False, "use_dictionary": False, "use_lemmatization": True, "use_stemming": False},
    #{"remove_numbers": False, "use_dictionary": False, "use_lemmatization": False, "use_stemming": True},
]

In [30]:
for params in parameter_sets:
    # Create a copy of the original DataFrame
    df_copy = df.copy()

    # Apply preprocessing with the current parameter combination
    df_copy['Text'] = preprocess_text_with_progress(df_copy, 'Text', **params)

    # Define the folder where CSV files will be saved
    output_folder = "processed_data"
    
    # Ensure the folder exists (create it if necessary)
    os.makedirs(output_folder, exist_ok=True)
    
    # Generate the full path for saving the file
    filename = os.path.join(output_folder, "Resume_processed_" + "_".join([f"{key}={value}" for key, value in params.items()]) + ".csv")
    
    # Save the DataFrame to the specified folder
    df_copy.to_csv(filename, index=False)

    print(f"Processed DataFrame saved as: {filename}")

Processing Resumes: 100%|██████████| 15873/15873 [02:54<00:00, 90.84it/s] 


Processed DataFrame saved as: processed_data\Resume_processed_remove_numbers=True_use_dictionary=True_use_lemmatization=True_use_stemming=False.csv
